In [95]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import pandas as pd
from core import semcor_bert_pipeline

In [209]:
shared_metrics = {}
shared_words = ['foot.n', 'table.n', 'plane.n', 'right.n', 'model.n', 'degree.n']
fmt_sense_12 = lambda w: [w + '.01', w + '.02']
shared_senses = [fmt_sense_12(w) for w in shared_words]
shared_senses[-1] = ['degree.n.01', 'academic_degree.n.01']
shared_senses[2] = ['airplane.n.01', 'plane.n.02']
for w_s in zip(shared_words, shared_senses):
    model_data = binary_logistic(w_s[0], w_s[1])
    weight_values, weight_indices = nonzero_weights(model_data['model'])
    f_scores, accuracies = k_fold_cv(model_data['data'], model_data['transformed_labels'])
    shared_metrics[w_s[0]] = {'senses': w_s[1], 'data': model_data, 'weights': weight_values,
                    'weight_indices': weight_indices, 'f1_kfold': f_scores, 'acc_kfold': accuracies}
    

/anaconda3/envs/bert/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/anaconda3/envs/bert/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/anaconda3/envs/bert/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/anaconda3/envs/bert/lib/python3.7/site-packages/sklearn/metr

In [223]:
shared_stats = []
for k in shared_metrics:
    word_results = shared_metrics[k]
    weights = word_results['weights']
    shared_stats.append({'word_type': k, 'avg_f1': np.mean(word_results['f1_kfold']), 'avg_acc': np.mean(word_results['acc_kfold']),
    'pct_nonzero_weights': len(weights) / 768, 'max_wt': max(weights), 'min_wt': min(weights), 'mean_nonzero_wt': np.mean(weights), 'sd_nonzero_wt': np.std(weights)})
pd.DataFrame(shared_stats)

,word_type,avg_f1,avg_acc,pct_nonzero_weights,max_wt,min_wt,mean_nonzero_wt,sd_nonzero_wt
0,foot.n,0.600000,1.000000,0.027344,0.299130,-0.370585,0.000001,0.162309
1,table.n,0.400000,1.000000,0.023438,0.235134,-0.218122,0.026061,0.135472
2,plane.n,0.353846,0.925000,0.019531,0.455377,-0.292556,0.030015,0.191023
3,right.n,0.310000,0.900000,0.013021,0.183585,-0.700916,-0.048585,0.239278
4,model.n,0.350000,0.833333,0.022135,0.393248,-0.229641,0.028511,0.162248
5,degree.n,0.800000,1.000000,0.016927,0.310833,-0.240271,-0.022812,0.146687


In [212]:
[(k, shared_metrics[k]['weight_indices']) for k in shared_words]

[('foot.n',
  array([ 13,  29,  70,  89, 141, 158, 191, 226, 231, 287, 304, 308, 493,
         518, 532, 547, 637, 664, 693, 709, 730])),
 ('table.n',
  array([  9,  37,  49, 136, 143, 286, 308, 317, 332, 334, 393, 398, 445,
         471, 609, 619, 666, 680])),
 ('plane.n',
  array([105, 157, 254, 286, 308, 409, 411, 414, 448, 480, 513, 619, 637,
         685, 695])),
 ('right.n', array([ 58,  62,  65, 135, 184, 308, 338, 514, 606, 747])),
 ('model.n',
  array([ 74, 121, 161, 282, 308, 340, 364, 423, 434, 450, 452, 473, 525,
         552, 586, 619, 739])),
 ('degree.n',
  array([ 15,  22, 220, 262, 328, 350, 432, 523, 541, 544, 565, 620, 724]))]